In [1]:
import random

import numpy as np
import pandas as pd
import seaborn as sns

from algorithm_gd import forward, loss_fn, r2_score, gradient_m, gradient_c, get_iteration_vs_accuracy_data

In [2]:
df = pd.read_csv("SOCR-HeightWeight.csv")
df.rename(columns={"Height(Inches)": "X", "Weight(Pounds)": "y"}, inplace=True)
df

,Index,X,y
0,1,65.78331,112.9925
1,2,71.51521,136.4873
2,3,69.39874,153.0269
3,4,68.21660,142.3354
4,5,67.78781,144.2971
...,...,...,...
24995,24996,69.50215,118.0312
24996,24997,64.54826,120.1932
24997,24998,64.69855,118.2655
24998,24999,67.52918,132.2682


In [3]:
no_of_nodes = 4

In [4]:
node_data_partitions = np.array_split(df, no_of_nodes)
node_data_partitions

/home/agaru/anaconda3/envs/cvf/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[      Index         X         y
 0         1  65.78331  112.9925
 1         2  71.51521  136.4873
 2         3  69.39874  153.0269
 3         4  68.21660  142.3354
 4         5  67.78781  144.2971
 ...     ...       ...       ...
 6245   6246  68.55917  131.6865
 6246   6247  69.82956  152.9133
 6247   6248  68.56366  135.9443
 6248   6249  69.89716  135.4161
 6249   6250  67.22911  126.9908
 
 [6250 rows x 3 columns],
        Index         X          y
 6250    6251  68.27784  151.49860
 6251    6252  67.70956  120.92840
 6252    6253  68.63497  143.90900
 6253    6254  66.24494  148.30150
 6254    6255  65.92820   98.38391
 ...      ...       ...        ...
 12495  12496  68.03870  138.32280
 12496  12497  69.78338  124.38070
 12497  12498  67.94391  131.20250
 12498  12499  66.63849  126.91320
 12499  12500  68.45079  127.41400
 
 [6250 rows x 3 columns],
        Index         X         y
 12500  12501  70.07106  113.5241
 12501  12502  69.31026  114.3171
 12502  12503  71.75837  1

In [5]:
for i, node_data in enumerate(node_data_partitions):
    df.loc[node_data.index, "node"] = i

df["node"] = df["node"].astype("int")
df

,Index,X,y,node
0,1,65.78331,112.9925,0
1,2,71.51521,136.4873,0
2,3,69.39874,153.0269,0
3,4,68.21660,142.3354,0
4,5,67.78781,144.2971,0
...,...,...,...,...
24995,24996,69.50215,118.0312,3
24996,24997,64.54826,120.1932,3
24997,24998,64.69855,118.2655,3
24998,24999,67.52918,132.2682,3


In [6]:
df['X_sqr'] = df['X'].apply(lambda x: np.square(x))
df['Xy'] = df[['X', 'y']].apply(lambda row: row.X * row.y, axis=1)

In [7]:
df

,Index,X,y,node,X_sqr,Xy
0,1,65.78331,112.9925,0,4327.443875,7433.020655
1,2,71.51521,136.4873,0,5114.425261,9760.917922
2,3,69.39874,153.0269,0,4816.185114,10619.874046
3,4,68.21660,142.3354,0,4653.504516,9709.637048
4,5,67.78781,144.2971,0,4595.187185,9781.584398
...,...,...,...,...,...,...
24995,24996,69.50215,118.0312,3,4830.548855,8203.422167
24996,24997,64.54826,120.1932,3,4166.477869,7758.261924
24997,24998,64.69855,118.2655,3,4185.902372,7651.606365
24998,24999,67.52918,132.2682,3,4560.190151,8931.963086


In [8]:
L = 0.001

In [9]:
doubly_stochastic_matrix_config = [
    [1 / 2, 1 / 4, 1 / 8, 1 / 8],
    [1 / 4, 3 / 4, 0, 0],
    [1 / 8, 0, 7 / 8, 0],
    [1 / 8, 0, 0, 7 / 8],
]

In [32]:
inv = [1.9, 1.9, 1.9, 1.9]
s0 = [0.0, 0.0, 0.0, 0.0]
s1 = [1.7, 1.9, 1.1, 1.8]
s2 = [1.7, 1.9, 1.1, 1.9]

In [39]:
def get_node_df(pos):
    df_copy = df[df["node"]==pos]
    return df_copy

In [35]:
def get_front(state, pos):
    dbl_st_mat = doubly_stochastic_matrix_config[pos]
    return sum(i*j for i, j in zip(dbl_st_mat, state))

In [49]:
def get_p(pos):
    df_copy = get_node_df(pos)
    return -2/df_copy.Index.count()

In [42]:
def get_q(pos):
    df_copy = get_node_df(pos)
    return np.sum(df_copy['Xy'])

In [41]:
def get_r(pos):
    df_copy = get_node_df(pos)
    return np.sum(df_copy['X_sqr'])

In [15]:
p = get_p()
q = get_q()
r = get_r()
p, q, r

(np.float64(-8e-05),
 np.float64(216291902.5483566),
 np.float64(115666993.35202293))

In [16]:
def get_back(s, pos):
    m = s[pos]
    df_copy = df[df["node"]==pos]
    return np.sum(df_copy['Xy'].array - m * df_copy['X_sqr']) 

In [52]:
s = s0[:]
s_copy = s[:]
print(s)
for i in range(10):
    for pos in range(len(s)):
    # for pos in [3]:
        p = get_p(pos)
        frnt = get_front(s_copy, pos)
        back = get_back(s_copy, pos)
        # s[pos] = frnt - L * p * (q - s[pos] * r)
        s[pos] = frnt - L * p * back
        # print(frnt, s)
    s_copy = s[:]
    print(s)

[0.0, 0.0, 0.0, 0.0]
0.0 [np.float64(17.31597725088188), 0.0, 0.0, 0.0]
0.0 [np.float64(17.31597725088188), np.float64(17.30768634843994), 0.0, 0.0]
0.0 [np.float64(17.31597725088188), np.float64(17.30768634843994), np.float64(17.28257483077795), 0.0]
0.0 [np.float64(17.31597725088188), np.float64(17.30768634843994), np.float64(17.28257483077795), np.float64(17.307170385374352)]
[np.float64(17.31597725088188), np.float64(17.30768634843994), np.float64(17.28257483077795), np.float64(17.307170385374352)]
17.308628364569962 [np.float64(-125.49219448508995), np.float64(17.30768634843994), np.float64(17.28257483077795), np.float64(17.307170385374352)]
17.309759074050426 [np.float64(-125.49219448508995), np.float64(-125.52430632967213), np.float64(17.28257483077795), np.float64(17.307170385374352)]
17.28675013329094 [np.float64(-125.49219448508995), np.float64(-125.52430632967213), np.float64(-125.33070747613148), np.float64(17.307170385374352)]
17.308271243562793 [np.float64(-125.4921944850

In [28]:
s = s1[:]
s_copy = s[:]
print(s)
for i in range(5):
    for pos in range(len(s)):
    # for pos in [3]:
        frnt = get_front(s_copy, pos)
        back = get_back(s, pos)
        # s[pos] = frnt - L * p * (q - s[pos] * r)
        s[pos] = frnt - L * p * back
        # print(frnt, s)
    s_copy = s[:]
    print(s)

[1.7, 1.9, 1.1, 1.8]
[np.float64(2.086618507457178), np.float64(1.7819186646579237), np.float64(2.9513174512418923), np.float64(1.946420468200421)]
[np.float64(1.6063818917964376), np.float64(2.0631532447080123), np.float64(0.3374060797619296), np.float64(1.7838313644178216)]
[np.float64(2.19966864409234), np.float64(1.5034796041384368), np.float64(4.036243447756171), np.float64(1.9580088738700505)]
[np.float64(1.4690244383487634), np.float64(2.5266604227971614), np.float64(-1.208612132789841), np.float64(1.7812697125692176)]
[np.float64(2.370822307062269), np.float64(0.7446046231437389), np.float64(6.242284858410029), np.float64(1.944529212011453)]
